### Current Process
1. Read in data --> Done

2. Custom Imputation --> Done

3. Aggregate Data --> Not Done Yet

4. Add Binary Class --> Done, Should Add Binary Class Later

5. Summary Statistics Features --> Done

6. Wrapper Functions --> Done, Need to Test Though

7. Sklearn Pipeline Categorical Features --> One Hot Encoding Done

8. Sklearn Pipeline Numerical Features --> StandardScaler Done

9. Create Lagged Features --> Done

10. Modeling --> Currently XgBoost, (Maybe Try: TensorFlow Decision Tree, TensorFlow Probability Model)

11. Model Evaluation --> Accuracy, Precision, Recall, F1, Confusion Matrix (Need to add Variable Importance Based on Variance)

12. PySpark: XGBoost Classification Feature Importance

In [1]:
from Input_Variables.read_vars import raw_data_storage, \
                                      analysis_group, \
                                      daily_stats_features_lower, daily_stats_features_upper, \
                                      ml_models_train_split, ml_models_test_split, model_storage_location, \
                                      time_series_lag_values_created

from Data_Schema.schema import Pandas_UDF_Data_Schema
from Read_In_Data.read_data import Reading_Data
from Data_Pipeline.sklearn_pipeline import Sklearn_Pipeline
from Feature_Generation.create_binary_labels import Create_Binary_Labels
from Feature_Generation.summary_stats import Summary_Stats_Features
from Feature_Generation.lag_features import Create_Lagged_Features
from Model_Creation.xgboost_model import XGBoost_Classification
from Model_Evaluation.classification_evaluation import Classification_Evalaution_Metrics
from Model_Plots.xgboost_classification_plots import XGBoost_Classification_Plot

# General Modules

In [2]:
# PySpark UDF Schema Activation
pandas_udf_data_schema=Pandas_UDF_Data_Schema()

# Data Location
reading_data=Reading_Data(data_location=raw_data_storage)

# Create Binary y Variables
create_binary_labels=Create_Binary_Labels()

# Sklearn Pipeline
pandas_sklearn_pipeline=Sklearn_Pipeline()

# Features Daily Stats Module
summary_stats_features=Summary_Stats_Features()

# Features Lagged Value
create_lag_features=Create_Lagged_Features()

# XGBoost Model Module
xgboost_classification=XGBoost_Classification()

# Classification Evaluation
classification_evalaution_metrics=Classification_Evalaution_Metrics()

# Model Plots Feature Importance
xgboost_classification_plot=XGBoost_Classification_Plot()

# PySpark

### 1. PySpark: Reading In Data

In [3]:
####### PySpark
pyspark_df=reading_data.read_in_pyspark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/18 18:48:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
pyspark_df.show(1)

+--------------------+-----+-------------------+---------------------+------------------+
|           PatientId|Value| GlucoseDisplayTime|GlucoseDisplayTimeRaw|GlucoseDisplayDate|
+--------------------+-----+-------------------+---------------------+------------------+
|b+SG81ztOyK5NQEu3...|  0.0|2022-09-11 11:34:07| 2022-09-11T11:34:...|        2022-09-11|
+--------------------+-----+-------------------+---------------------+------------------+
only showing top 1 row



### 2. PySpark: Custom Imputation Pipeline

In [7]:
from pyspark.sql.functions import pandas_udf, PandasUDFType, lit
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, FloatType

In [8]:
test_data_schema = StructType([StructField('PatientId', StringType(), True),
                               StructField('GlucoseDisplayTime', TimestampType(), True),
                               StructField('Value', FloatType(), True)])

In [10]:
def pyspark_custom_imputation_pipeline(df, output_schema, analysis_group):
    @pandas_udf(output_schema, PandasUDFType.GROUPED_MAP)
    def transform_features(pdf):
        df=pdf[['PatientId', 'Value', 'GlucoseDisplayTime']]

#             # Imputation
#             custom_imputation=Pipeline(steps=[("custom_imputation",
#                                        FunctionTransformer(self.value_imputation.fill_missing_bootstrap))])

#             transformed_data1=custom_imputation.fit_transform(df)
#             transformed_data_df=pd.DataFrame(transformed_data1)

#             return transformed_data_df
        return df

    # df=df.withColumn("Group_Col", lit(1))

    transformed_data=df.groupby(analysis_group).apply(transform_features)

    return transformed_data

In [11]:
analysis_group

'PatientId'

In [13]:
vals=pyspark_custom_imputation_pipeline(df=pyspark_df,
                                  output_schema=test_data_schema,
                                  analysis_group=analysis_group)

In [15]:
vals.show(1)

23/04/18 18:54:25 ERROR Executor: Exception in task 0.0 in stage 4.0 (TID 5)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "pyarrow/array.pxi", line 830, in pyarrow.lib._PandasConvertible.to_pandas
  File "pyarrow/table.pxi", line 437, in pyarrow.lib.ChunkedArray._to_pandas
  File "pyarrow/array.pxi", line 1649, in pyarrow.lib._array_like_to_pandas
  File "pyarrow/pandas-shim.pxi", line 103, in pyarrow.lib._PandasAPIShim.series
  File "pyarrow/pandas-shim.pxi", line 100, in pyarrow.lib._PandasAPIShim._check_import
  File "pyarrow/pandas-shim.pxi", line 52, in pyarrow.lib._PandasAPIShim._import_pandas
  File "pyarrow/pandas-shim.pxi", line 47, in pyarrow.lib._PandasAPIShim._import_pandas
ModuleNotFoundError: No module named 'pandas'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:101)
	at 

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "pyarrow/array.pxi", line 830, in pyarrow.lib._PandasConvertible.to_pandas
  File "pyarrow/table.pxi", line 437, in pyarrow.lib.ChunkedArray._to_pandas
  File "pyarrow/array.pxi", line 1649, in pyarrow.lib._array_like_to_pandas
  File "pyarrow/pandas-shim.pxi", line 103, in pyarrow.lib._PandasAPIShim.series
  File "pyarrow/pandas-shim.pxi", line 100, in pyarrow.lib._PandasAPIShim._check_import
  File "pyarrow/pandas-shim.pxi", line 52, in pyarrow.lib._PandasAPIShim._import_pandas
  File "pyarrow/pandas-shim.pxi", line 47, in pyarrow.lib._PandasAPIShim._import_pandas
ModuleNotFoundError: No module named 'pandas'


In [ ]:
####### PySpark
pyspark_custom_imputation_schema=pandas_udf_data_schema.custom_imputation_pyspark_schema()
pyspark_custom_imputation_pipeline=pandas_sklearn_pipeline.pyspark_custom_imputation_pipeline(df=pyspark_df, 
                                                                                              output_schema=pyspark_custom_imputation_schema,
                                                                                              analysis_group=analysis_group)
pyspark_custom_imputation_pipeline.show(1)

### 3. PySpark Aggregate Data at Level

### 4. PySpark: Adding Binary Labels

In [ ]:
pyspark_df_added_binary_labels=create_binary_labels.pyspark_binary_labels(df=pyspark_custom_imputation_pipeline, 
                                                                          lower=daily_stats_features_lower, 
                                                                          upper=daily_stats_features_upper)

pyspark_df_added_binary_labels.show(1)

### 5. PySpark: Features: Summary Statistics

In [ ]:
####### PySpark
pyspark_features_summary_stats_schema=pandas_udf_data_schema.summary_stats_schema()
pyspark_features_summary_stats=summary_stats_features.pyspark_summary_statistics(df=pyspark_df_added_binary_labels, 
                                                                                 output_schema=pyspark_features_summary_stats_schema, 
                                                                                 lower=daily_stats_features_lower, 
                                                                                 upper=daily_stats_features_upper,
                                                                                 analysis_group=analysis_group)

pyspark_features_summary_stats.show(1)

### 6. PySpark: Wrapper Functions

### 7. PySpark: Sklearn Categorical Pipeline in PySpark

In [ ]:
####### PySpark
pyspark_categorical_schema=pandas_udf_data_schema.sklearn_pyspark_categorical_schema()
pyspark_transformations_categorical=pandas_sklearn_pipeline.pyspark_sklearn_pipeline_categorical(df=pyspark_features_summary_stats, 
                                                                                                 output_schema=pyspark_categorical_schema,
                                                                                                 analysis_group=analysis_group)
pyspark_transformations_categorical.show(1)

### 8. PySpark: Sklearn Numerical Pipeline in PySpark

In [ ]:
####### PySpark
pyspark_numerical_schema=pandas_udf_data_schema.sklearn_pyspark_numerical_schema()
pyspark_transformations_numerical=pandas_sklearn_pipeline.pyspark_sklearn_pipeline_numerical(df=pyspark_transformations_categorical, 
                                                                                             output_schema=pyspark_numerical_schema)
pyspark_transformations_numerical.show(1)

### 9. PySpark: Lag Features

In [ ]:
####### PySpark
pyspark_lag_features_creation=create_lag_features.pyspark_lag_features(df=pyspark_transformations_numerical,
                                                                       time_series_lag_values_created=time_series_lag_values_created)
pyspark_lag_features_creation.show(5)

### 10. PySpark: XGBoost Model

In [ ]:
####### PySpark
pyspark_xgboost_classsification_schema=pandas_udf_data_schema.xgboost_classification_schema()

classification_model_outputs=xgboost_classification.pyspark_xgboost(df=pyspark_lag_features_creation, 
                                                                    output_schema=pyspark_xgboost_classsification_schema, 
                                                                    train_split=ml_models_train_split, 
                                                                    test_split=ml_models_test_split)

classification_model_outputs.show()

### 11. PySpark: Model Evaluation

In [ ]:
####### PySpark
pyspark_classification_metric_schema=pandas_udf_data_schema.classification_metric_schema()

classification_metric_df=classification_evalaution_metrics.pyspark_classification_model_evaluation_metrics(df=classification_model_outputs, 
                                                                                                           output_schema=pyspark_classification_metric_schema)
classification_metric_df.show()                    

### 12. PySpark: XGBoost Classification Feature Importance

In [ ]:
xgboost_classification_plot.read_model_plot_variance(model_storage_location=model_storage_location)

# Pandas

### Pandas: Reading in Data

In [ ]:
####### Pandas
pandas_df=reading_data.read_in_pandas()
pandas_df.head(1)

### Pandas: Custom Imputation Pipeline

In [ ]:
test_pat=pandas_df[pandas_df['PatientId']=='tHu8WPnIffml5CL+AbOBkXcbFApQnP06KdrHbjinta4=']

In [ ]:
pandas_custom_imputation_pipeline=pandas_sklearn_pipeline.pandas_custom_imputation_pipeline(df=pandas_df)
pandas_custom_imputation_pipeline.head(1)

In [ ]:
####### Pandas
pandas_custom_imputation_pipeline=pandas_sklearn_pipeline.pandas_custom_imputation_pipeline(df=pandas_df)
pandas_custom_imputation_pipeline.head(1)

### Pandas: Aggregate Data at Level

### Pandas: Adding Binary Labels

In [ ]:
pandas_df_added_binary_labels=create_binary_labels.pandas_binary_labels(df=pandas_custom_imputation_pipeline, 
                                                                        lower=daily_stats_features_lower, 
                                                                        upper=daily_stats_features_upper)
pandas_df_added_binary_labels.head(1)

### Pandas: Features: Summary Statistics

In [ ]:
pandas_features_summary_stats=summary_stats_features.pandas_compressDailyValues(data=pandas_df_added_binary_labels, 
                                                                                lower=daily_stats_features_lower, 
                                                                                upper=daily_stats_features_upper)
pandas_features_summary_stats.head(1)

### Pandas: Wrapper Functions

### Pandas: Sklearn Categorical Pipeline in Pandas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
df=pandas_features_summary_stats[['PatientId', 'Value', 'GlucoseDisplayTime', 'GlucoseDisplayDate', 'inserted', 
        'missing', 'y_Binary', 'Median', 'Mean', 'Std Dev', 'Max', 'Min', 'AreaBelow', 'AreaAbove']]

In [ ]:
df.head()

In [ ]:
for patient_id in df['PatientId'].unique():
    # Categorical Features
    categorical_features=['inserted', 'missing']
    categorical_transformer=Pipeline([('imputer_cat', SimpleImputer(strategy='constant', fill_value=np.nan)),
                                        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor_2=ColumnTransformer([('categorical', categorical_transformer, categorical_features)],
                                    remainder = 'passthrough')

    cat_pipe_pipeline=Pipeline([('preprocessing_2', preprocessor_2)])

    transformed_data1=cat_pipe_pipeline.fit_transform(df)

    transformed_data_df=pd.DataFrame(transformed_data1)

    transformed_data_df['combine_inserted']=transformed_data_df[[0,1]].values.tolist()
    transformed_data_df['combine_missing']=transformed_data_df[[2,3]].values.tolist()
    transformed_data_df=transformed_data_df.drop(transformed_data_df.iloc[:, 0:4],axis = 1)

    transformed_data_df.columns=['PatientId', 'Value', 'GlucoseDisplayTime', 'GlucoseDisplayDate', 
                                    'y_Binary', 'Median', 'Mean', 'Std Dev', 'Max', 'Min', 'AreaBelow', 
                                    'AreaAbove', 'inserted', 'missing']

In [ ]:
transformed_data_df.columns

In [ ]:
####### Pandas
pandas_custom_categorical_pipeline=pandas_sklearn_pipeline.pandas_transform_categorical_features(df=pandas_features_summary_stats)
pandas_custom_categorical_pipeline.head(1)

### Pandas: Sklearn Numerical Pipeline in Pandas

In [ ]:
####### Pandas
pandas_custom_numerical_pipeline=pandas_sklearn_pipeline.pandas_transform_numerical_features(df=pandas_custom_categorical_pipeline)
pandas_custom_numerical_pipeline.head(1)

### Pandas: Models